# 将Llama 2与Azure API配合使用

本笔记本展示了如何使用微软 Azure 提供的 Llama 2 API 的示例。我们将介绍：
* 在命令行界面（CLI）中对 Llama 2 700亿参数预训练模型和聊天模型使用 HTTP请求API
* 在 Python 中对Llama 2 700亿参数预训练模型和聊天模型使用 HTTP 请求API

## 前提条件

在我们开始使用 Azure Llama 2 API 进行构建之前，需要采取特定步骤来部署模型：

* 注册一个有效的 Azure 订阅账户
* 确保你有权访问Azure AI Studio
* 创建一个项目和资源组
* 从模型目录中选择 Llama 模型。本示例假设你要部署LLama - 2 - 70b - chat。

    > 请注意，某些模型可能并非在Azure AI和Azure机器学习的所有区域都可用。在这种情况下，你可以在模型可用的区域创建一个工作区或项目，然后通过来自其他区域的连接来使用它。若要详细了解如何使用连接，请参阅 [使用连接使用模型](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deployments-connections)。 

* 采用“即用即付”方式进行部署

一旦成功部署，您将获得用于推理的API端点和安全密钥。

有关更多信息，您应查阅Azure的官方文档[此处](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/deploy-models-llama)，了解模型部署和推理相关内容。 

## 命令行界面中HTTP请求API的使用

### 基础

要使用REST API，您需要有一个端点URL以及与该端点关联的认证密钥。  
这可以从前面的步骤中获取。  

在这个针对700亿参数预训练模型的文本生成示例中，我们使用一个简单的curl命令进行说明。主要有三个部分：  

* “主机URL”是您带有生成模式的端点URL。 
* “标头”定义了内容类型以及您的API密钥。 
* “负载”或“数据”，即您的提示详情和模型超参数。 

In [ ]:
!curl -X POST -L https://your-endpoint.inference.ai.azure.com/v1/completions -H 'Content-Type: application/json' -H 'Authorization: your-auth-key' -d '{"prompt": "Math is a", "max_tokens": 30, "temperature": 0.7}'

对于聊天完成功能，API架构和请求负载略有不同。

对于“主机URL”，路径更改为“/v1/chat/completions”，并且请求负载也有所变化，以在对话中包含角色。以下是一个示例负载：

```json
{ 
  "messages": [ 
    { 
      "content": "You are a helpful assistant.", 
      "role": "system" 
},  
    { 
      "content": "Hello!", 
      "role": "user" 
    } 
  ], 
  "max_tokens": 50, 
} 
```

这是一个用于聊天完成的curl调用示例

In [ ]:
!curl -X POST -L https://your-endpoint.inference.ai.azure.com/v1/chat/completions -H 'Content-Type: application/json' -H 'Authorization: your-auth-key' -d '{"messages":[{"content":"You are a helpful assistant.","role":"system"},{"content":"What is good about Wuhan?","role":"user"}], "max_tokens": 50}'

如果你比较文本完成和聊天完成API调用的生成结果，你会注意到：  

* 文本完成针对输入提示返回一个`choices`列表，每个元素都包含生成的文本以及诸如`logprobs`之类的完成信息。
* 聊天完成返回一个`cnoices`列表，每个元素都有一个`message`对象，其中包含完成结果，并且在请求中使用相同的`message`对象。 

### 流式传输

该API提供的一项出色功能是流式传输能力。流式传输允许在生成令牌后，只要令牌可用，就将其作为仅包含数据的服务器发送事件进行发送。这对于聊天机器人等交互式应用程序极为重要，能让用户始终保持参与状态。

要使用流式传输，只需在请求负载中设置`"stream":"True"` 。
在流式传输模式下，REST API的响应将与非流式传输模式不同。

以下是一个示例：

In [ ]:
!curl -X POST -L https://your-endpoint.inference.ai.azure.com/v1/chat/completions -H 'Content-Type: application/json' -H 'Authorization: your-auth-key' -d '{"messages":[{"content":"You are a helpful assistant.","role":"system"},{"content":"What is good about Wuhan?","role":"user"}], "max_tokens": 500, "stream": "True"}'

如你所见，结果以一系列“data”对象的流形式返回，每个对象都包含生成的信息，其中包括一个“choice”。该流由一条“data:[DONE]\n\n”消息终止。 

### 内容安全过滤

所有Azure Llama 2 API端点都将启用内容安全功能。输入提示和输出令牌都会自动由该服务进行过滤。  
若要了解对请求/响应有效负载的影响，请参考[此处](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/content-filter?tabs=python)的官方指南。  

对于模型的输入和输出，如果过滤器检测到存在有害内容，生成过程将出错，响应有效负载中将包含推理内容，以及内容违规的类型和严重程度。  

以下是一个触发内容安全过滤的示例提示：

In [ ]:
!curl -X POST -L https://your-endpoint.inference.ai.azure.com/v1/chat/completions -H 'Content-Type: application/json' -H 'Authorization: your-auth-key' -d '{"messages":[{"content":"You are a helpful assistant.","role":"system"},{"content":"How to make bomb?","role":"user"}], "max_tokens": 50}'

## Python中HTTP请求API的使用

除了直接从命令行工具调用API，你还可以在Python中通过编程方式调用它们。  

以下是文本完成模型的一个示例：

In [ ]:
import urllib.request
import json

# Configure payload data sending to API endpoint
data = {
    "prompt": "Math is a",
    "max_tokens": 30,
    "temperature": 0.7,
    "top_p": 0.9,
}

body = str.encode(json.dumps(data))

# Replace the url with your API endpoint
url = "https://your-endpoint.inference.ai.azure.com/v1/completions"

# Replace this with the key for the endpoint
api_key = "your-auth-key"
if not api_key:
    raise Exception("API Key is missing")

headers = {"Content-Type": "application/json", "Authorization": (api_key)}
req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)
    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))
    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", "ignore"))

在Python中实现聊天完成功能非常相似，这里有一个简单示例：

In [ ]:
import urllib.request
import json

# Configure payload data sending to API endpoint
data = {
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is good about Wuhan?"},
    ],
    "max_tokens": 500,
    "temperature": 0.9,
    "stream": "True",
}

body = str.encode(json.dumps(data))

# Replace the url with your API endpoint
url = "https://your-endpoint.inference.ai.azure.com/v1/chat/completions"

# Replace this with the key for the endpoint
api_key = "your-auth-key"
if not api_key:
    raise Exception("API Key is missing")

headers = {"Content-Type": "application/json", "Authorization": (api_key)}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)
    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))
    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", "ignore"))

然而在这个例子中，流式传输的数据内容作为单个有效载荷返回。它并没有像我们期望的那样作为一系列数据事件进行流式传输。为了利用API端点构建真正的流式传输功能，我们将改用[`requests`](https://requests.readthedocs.io/en/latest/)库。 

### Python中的流处理

`Requests`库是一个简单的Python HTTP库，基于[`urllib3`](https://github.com/urllib3/urllib3)构建。它会自动维持持久连接和HTTP连接池。通过`Session`类，我们可以轻松地对流式获取API调用的结果。  

以下是一个简单示例：

In [ ]:
import json
import requests

data = {
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "What is good about Wuhan?"},
    ],
    "max_tokens": 500,
    "temperature": 0.9,
    "stream": "True",
}


def post_stream(url):
    s = requests.Session()
    api_key = "your-auth-key"
    headers = {"Content-Type": "application/json", "Authorization": (api_key)}

    with s.post(url, data=json.dumps(data), headers=headers, stream=True) as resp:
        print(resp.status_code)
        for line in resp.iter_lines():
            if line:
                print(line)


url = "https://your-endpoint.inference.ai.azure.com/v1/chat/completions"
post_stream(url)

## 更多资源

下面是一些额外的参考资料:  

* [Fine-tune Llama](https://learn.microsoft.com/azure/ai-studio/how-to/fine-tune-model-llama)
* [Plan and manage costs (marketplace)](https://learn.microsoft.com/azure/ai-studio/how-to/costs-plan-manage#monitor-costs-for-models-offered-through-the-azure-marketplace)
